In [0]:
!pip3 uninstall pytorch-hrvvi-ext
!pip3 install -U --no-cache-dir --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple pytorch-hrvvi-ext

In [0]:
import sys
import os

import torch
import hutil
import matplotlib.pyplot as plt
print(hutil.__version__)

1.4.4


In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
gdrive = "/gdrive"
from google.colab import drive
drive.mount(gdrive, force_remount=True)
mydrive = os.path.join(gdrive, "My Drive")
!ls /gdrive/My\ Drive

def gpath(p):
    return os.path.join(mydrive, p)

Mounted at /gdrive
'Colab Notebooks'   eng-fra.pt	 images   repo	   weixin.pkl
 datasets	    fonts	 models   result


In [0]:
from PIL import Image
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import LambdaLR, MultiStepLR

from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, RandomHorizontalFlip

from hutil.data import train_test_split, Fullset
from hutil.ext.summary import summary
from hutil.train import Trainer, init_weights
from hutil.train.metrics import Accuracy, TrainLoss
from hutil.transformers import InputTransform


In [0]:
train_transforms = InputTransform(
    Compose([
        RandomCrop(32, padding=4),
        RandomHorizontalFlip(),
        ToTensor(),
        Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    ])
)

test_transform = InputTransform(
    Compose([
        ToTensor(),
        Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    ])
)

data_home = gpath("datasets/CIFAR10")
ds = CIFAR10(data_home, train=True, download=True)
ds_train, ds_val = train_test_split(
    ds, test_ratio=0.04,
    transform=train_transforms,
    test_transform=test_transform,
)
ds_test = Fullset(CIFAR10(data_home, train=False,
                          download=True), test_transform)


Files already downloaded and verified
Files already downloaded and verified


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)


class SELayer(nn.Module):
    def __init__(self, in_channels, reduction=8):
        super().__init__()
        channels = in_channels // reduction
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.layers = nn.Sequential(
            nn.Linear(in_channels, channels),
            nn.ReLU(True),
            nn.Linear(channels, in_channels),
            nn.Sigmoid(),
        )

    def forward(self, x):
        b, c = x.size()[:2]
        s = self.avgpool(x).view(b, c)
        s = self.layers(s).view(b, c, 1, 1)
        return x * s


class WideSEBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, with_se=False):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = conv3x3(in_channels, out_channels, stride=stride)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)

        self.se = None
        if with_se:
            self.se = SELayer(out_channels)

        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Conv2d(
                in_channels, out_channels, 1, stride=stride)

    def forward(self, x):
        residual = x
        o1 = self.relu1(self.bn1(x))
        z = self.conv1(o1)
        o2 = self.relu2(self.bn2(z))
        z = self.conv2(o2)
        if self.se:
            z = self.se(z)
        if self.downsample:
            residual = self.downsample(o1)
        return z + residual


class ResNet(nn.Module):
    stages = [16, 16, 32, 64]

    def __init__(self, block, layers, k=4, num_classes=10, **kwargs):
        super().__init__()
        self.block_kwargs = kwargs
        self.conv = nn.Conv2d(
            3, self.stages[0], kernel_size=3, stride=1, padding=1)

        self.layer1 = self._make_layer(
            block, self.stages[0] * 1, self.stages[1] * k, layers[0], stride=1, **kwargs)
        self.layer2 = self._make_layer(
            block, self.stages[1] * k, self.stages[2] * k, layers[1], stride=2, **kwargs)
        self.layer3 = self._make_layer(
            block, self.stages[2] * k, self.stages[3] * k, layers[2], stride=2, **kwargs)

        self.bn = nn.BatchNorm2d(self.stages[3] * k)
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(self.stages[3] * k, num_classes)

    def _make_layer(self, block, in_channels, out_channels, blocks, stride=1, **kwargs):
        layers = []
        layers.append(block(in_channels, out_channels,
                            stride=stride, **kwargs))
        for i in range(1, blocks):
            layers.append(
                block(out_channels, out_channels, **kwargs))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.bn(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


In [0]:
net = ResNet(WideSEBasicBlock, [4,4,4], k=2)
net.apply(init_weights(nonlinearity='relu'))
criterion = nn.CrossEntropyLoss()
optimizer = SGD(net.parameters(), lr=1e-1, momentum=0.9, dampening=0, weight_decay=5e-4, nesterov=True)
lr_scheduler = MultiStepLR(optimizer, [30, 60, 80], gamma=0.2)

# optimizer = Adam(net.parameters(), lr=0.001, weight_decay=1e-4)
# lr_scheduler = LambdaLR(optimizer, lambda x: 0.98 ** x)


metrics = {
    'loss': TrainLoss(),
    'acc': Accuracy(),
}
trainer = Trainer(net, criterion, optimizer, lr_scheduler, metrics=metrics, save_path=gpath("models"), name="CIFAR10-WRN28-2")


In [0]:
lr_scheduler.milestones = [30, 70, 100, 110]

In [0]:
trainer.load_state_dict(torch.load(gpath("models/CIFAR10-WRN28-2_trainer_190.pth")))

In [0]:
summary(net, (3,32,32))

In [0]:

train_loader = DataLoader(ds_train, batch_size=32, shuffle=True, num_workers=1, pin_memory=True)
test_loader = DataLoader(ds_test, batch_size=128)
val_loader = DataLoader(ds_val, batch_size=128)



In [0]:
trainer.login_weixin(gpath("."))

In [26]:
trainer.fit(train_loader, 10, val_loader=val_loader, save_by_metric='-val_loss')

Epoch 111/120
elapsed: 134s	loss: 0.0262	acc: 0.9938	
validate ------	loss: 0.1958	acc: 0.9425	
Epoch 112/120
elapsed: 134s	loss: 0.0244	acc: 0.9943	
validate ------	loss: 0.1914	acc: 0.9450	
Epoch 113/120
elapsed: 134s	loss: 0.0226	acc: 0.9951	
validate ------	loss: 0.1924	acc: 0.9445	
Epoch 114/120
elapsed: 134s	loss: 0.0236	acc: 0.9948	
validate ------	loss: 0.1919	acc: 0.9460	
Epoch 115/120
elapsed: 134s	loss: 0.0216	acc: 0.9956	
validate ------	loss: 0.1939	acc: 0.9440	
Epoch 116/120
elapsed: 134s	loss: 0.0223	acc: 0.9951	
validate ------	loss: 0.1874	acc: 0.9465	
Epoch 117/120
elapsed: 134s	loss: 0.0214	acc: 0.9952	
validate ------	loss: 0.1900	acc: 0.9450	
Epoch 118/120
elapsed: 134s	loss: 0.0202	acc: 0.9960	
validate ------	loss: 0.1903	acc: 0.9460	
Epoch 119/120
elapsed: 134s	loss: 0.0201	acc: 0.9957	
validate ------	loss: 0.1964	acc: 0.9425	
Epoch 120/120
elapsed: 134s	loss: 0.0189	acc: 0.9959	
validate ------	loss: 0.1922	acc: 0.9455	


{'acc': [0.99375,
  0.9942708333333333,
  0.9950625,
  0.9947916666666666,
  0.9955625,
  0.9951458333333333,
  0.99525,
  0.9960208333333334,
  0.9956666666666667,
  0.9958958333333333],
 'loss': [0.026173482589423657,
  0.024427323937416076,
  0.022589903903504214,
  0.0235523734887441,
  0.021622312987844148,
  0.02231386575102806,
  0.021365505538880825,
  0.0202465183014671,
  0.020074109363059204,
  0.01886883933345477],
 'val_acc': [0.9425,
  0.945,
  0.9445,
  0.946,
  0.944,
  0.9465,
  0.945,
  0.946,
  0.9425,
  0.9455],
 'val_loss': [0.1958073754310608,
  0.19138987588882447,
  0.19235167717933654,
  0.19194952201843263,
  0.1938640127182007,
  0.18739703607559205,
  0.19002503490447997,
  0.1903388946056366,
  0.19640164184570313,
  0.1921942195892334]}

In [0]:
trainer.load_state_dict(torch.load(gpath("models/CIFAR10-WRN28-2_trainer_116_val_loss=0.187397.pth")))

In [31]:
trainer.evaluate(test_loader)


{'acc': 0.9374, 'loss': 0.22659484343528746}